In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# pip install psycopg2 


In [3]:
data_path = "Resources/mcd_menu.csv"
mcd_df = pd.read_csv(data_path)
mcd_df.head()

,Category,Item,Serving Size,Calories,Calories from Fat,Total Fat,Total Fat (% Daily Value),Saturated Fat,Saturated Fat (% Daily Value),Trans Fat,...,Carbohydrates,Carbohydrates (% Daily Value),Dietary Fiber,Dietary Fiber (% Daily Value),Sugars,Protein,Vitamin A (% Daily Value),Vitamin C (% Daily Value),Calcium (% Daily Value),Iron (% Daily Value)
0,Breakfast,Egg McMuffin,4.8 oz (136 g),300,120,13.0,20,5.0,25,0.0,...,31,10,4,17,3,17,10,0,25,15
1,Breakfast,Egg White Delight,4.8 oz (135 g),250,70,8.0,12,3.0,15,0.0,...,30,10,4,17,3,18,6,0,25,8
2,Breakfast,Sausage McMuffin,3.9 oz (111 g),370,200,23.0,35,8.0,42,0.0,...,29,10,4,17,2,14,8,0,25,10
3,Breakfast,Sausage McMuffin with Egg,5.7 oz (161 g),450,250,28.0,43,10.0,52,0.0,...,30,10,4,17,2,21,15,0,30,15
4,Breakfast,Sausage McMuffin with Egg Whites,5.7 oz (161 g),400,210,23.0,35,8.0,42,0.0,...,30,10,4,17,2,21,6,0,25,10


In [4]:
#clean table
new_mcd_df = mcd_df[['Item', 'Total Fat', 'Carbohydrates', 'Protein']].copy()
new_mcd_df.head()

,Item,Total Fat,Carbohydrates,Protein
0,Egg McMuffin,13.0,31,17
1,Egg White Delight,8.0,30,18
2,Sausage McMuffin,23.0,29,14
3,Sausage McMuffin with Egg,28.0,30,21
4,Sausage McMuffin with Egg Whites,23.0,30,21


In [5]:
# rename "Total Fat" column name to "total_fat"
new_mcd_df = new_mcd_df.rename(columns ={"Item":"item", "Total Fat": "total_fat", "Carbohydrates": "carbohydrates", "Protein": "protein" })
new_mcd_df.head()

,item,total_fat,carbohydrates,protein
0,Egg McMuffin,13.0,31,17
1,Egg White Delight,8.0,30,18
2,Sausage McMuffin,23.0,29,14
3,Sausage McMuffin with Egg,28.0,30,21
4,Sausage McMuffin with Egg Whites,23.0,30,21


In [10]:
#connect to local database (pgadmin)
rds_connection_string = "postgres:<password>@localhost:5432/fastfood"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
# check for tables created in pgadmin
engine.table_names()

['mcd_menu']

In [12]:
# Use pandas to load csv converted DataFrame into database
new_mcd_df.to_sql(name='mcd_menu', con=engine, if_exists='append', index=False)

In [13]:
# Confirm data has been added by querying the customer_name table
    # NOTE: can also check using pgAdmin
pd.read_sql_query('select * from mcd_menu', con=engine)

,item,calories,total_fat,carbohydrates,protein
0,Egg McMuffin,None,13,31,17
1,Egg White Delight,None,8,30,18
2,Sausage McMuffin,None,23,29,14
3,Sausage McMuffin with Egg,None,28,30,21
4,Sausage McMuffin with Egg Whites,None,23,30,21
...,...,...,...,...,...
515,McFlurry with Oreo Cookies (Small),None,17,80,12
516,McFlurry with Oreo Cookies (Medium),None,23,106,15
517,McFlurry with Oreo Cookies (Snack),None,11,53,8
518,McFlurry with Reese's Peanut Butter Cups (Medium),None,32,114,21
